In [ ]:
from evoVAE.utils.datasets import MSA_Dataset
import evoVAE.utils.seq_tools as st
import evoVAE.utils.metrics as mt
from evoVAE.models.seqVAETest import SeqVAETest
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
import numpy as np

#pd.set_option("display.max_rows", None)

This notebook can be used to test new features for a model without having to use the WandB service

# Config

In [ ]:
config = {
        # Dataset info
        "dataset": "playground",
        "seq_theta": 0.2, # reweighting 
        "AA_count": 21, # standard AA + gap
        
        # ADAM 
        "learning_rate": 1e-5, # ADAM
        "weight_decay": 0.01, # ADAM

        # Hidden units 
        "momentum": 0.1, 
        "dropout": 0.5,

        # Training loop 
        "epochs": 100,
        "batch_size": 128,
        "max_norm": 1.0, # gradient clipping
        
        # Model info
        "architecture": "SeqVAETest",
        "latent_dims": 3,
        "hidden_dims": [32, 16],
    }

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

# Read in data

In [ ]:
from pathlib import Path
DATA_PATH = "/Users/sebs_mac/uni_OneDrive/honours/data/gfp_alns/independent_runs/no_synthetic/ancestors/auto_rooted/ancestors/"
filepath = DATA_PATH + 'run_14_ancestors.fa'
aln = st.read_aln_file(filepath)
aln.head()

In [ ]:
import evoVAE.utils.seq_tools as st

msa, _, _ = st.convert_msa_numpy_array(aln)
msa.shape

In [ ]:
from numba import njit, prange, jit
import time
import numpy as np


In [ ]:

seqs = aln["sequence"].to_numpy()
subset = msa





In [ ]:
from evoVAE.utils.seq_tools import encode_and_weight_seqs


code, weight = encode_and_weight_seqs(aln['sequence'], 0.2)
aln['weight'] = weight

In [ ]:
aln.head()

In [ ]:

train, val = train_test_split(aln, test_size=0.2)


# TRAINING
train_dataset = MSA_Dataset(
    train["encoding"], train['weight'], train["id"]
)

# VALIDATION
val_dataset = MSA_Dataset(
    val["encoding"], val['weight'], val["id"]
)

# DATA LOADERS #
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=False)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

print(len(train_loader), len(val_loader))
#next(iter(train_loader))[0].shape

# Build model

In [ ]:
# get the sequence length 
SEQ_LEN = 0
BATCH_ZERO = 0
SEQ_ZERO = 0
seq_len = train_dataset[BATCH_ZERO][SEQ_ZERO].shape[SEQ_LEN]
input_dims = seq_len * config['AA_count']

seq_len, input_dims

# use preset structure for hidden dimensions 
model = SeqVAETest(input_dims=input_dims, latent_dims=config['latent_dims'], hidden_dims=config['hidden_dims'], config=config) 

model.eval()
for encoding, weights, _ in train_loader:

    encoding = encoding.float()
    weights = weights.float()
    #print(encoding.shape, weight.shape)
    modelOutputs = model(encoding)
    loss, kl, likelihood = model.loss_function(modelOutputs, encoding, weights)

In [ ]:
# very small nummy data 
dummy = next(iter(train_loader))[0].float()
print(dummy.shape)
# reconstruct input, note it has been flattened 
log_p, z_sample, z_mu, z_logvar = model(dummy)

# grab the shape of the input for reshaping
orig_shape = log_p.shape[0:-1]

# add on extra dim, then make it one-hot encoding shape (obs, seq_len, AA_count)
log_p = torch.unsqueeze(log_p, -1)
log_p = log_p.view(orig_shape + (-1, config['AA_count']))

In [ ]:
mut_data = pd.read_csv('../data/dms_data/GFP_AEQVI_Sarkisyan_2016.csv')
subset = mut_data.copy()[0:10]

In [ ]:
encoding, _ = st.encode_and_weight_seqs(mut_data['mutated_sequence'], 0.2, reweight=False)
mut_data['encoding'] = encoding
mut_data[0:10]

In [ ]:
mut_data.to_pickle("GFP_AEQVI_Sarkisyan_2016_dms_encoded.pkl")

In [ ]:
encoding, weights = st.encode_and_weight_seqs(subset['mutated_sequence'], 0.2, reweight=True)
subset['encoding'] = encoding
subset['weights'] = weights
subset

In [ ]:
metadata = pd.read_csv("../data/dms_data/DMS_substitutions.csv")
metadata = metadata[metadata["DMS_id"].str.contains("GFP")]
metadata


In [ ]:

import sklearn
import sklearn.preprocessing


wild_type = metadata['target_seq'].to_numpy()[0]
wild_one_hot = torch.Tensor(st.seq_to_one_hot(wild_type)).unsqueeze(0)

model.eval()
wild_model_encoding, _, _, _ = model(wild_one_hot)

orig_shape = wild_model_encoding.shape[0:-1]

wild_model_encoding = torch.unsqueeze(wild_model_encoding, -1)

wild_model_encoding = wild_model_encoding.view(orig_shape + (-1, model.AA_COUNT))


# get the wild type encoding 
wild_model_encoding = wild_model_encoding.squeeze(0)

one_hot = wild_one_hot.squeeze(0)
wt_prob = mt.seq_log_probability(one_hot, wild_model_encoding)

variant_encodings = torch.Tensor(np.stack(subset['encoding'].values))
variant_model_outputs, _, _, _ = model(variant_encodings)

model_scores =[]
for variant, var_one_hot in zip(variant_model_outputs, variant_encodings):
    
    print(variant.shape)
    var_model_encoding = torch.unsqueeze(variant, -1)
    print(var_model_encoding.shape)
    var_model_encoding = var_model_encoding.view(orig_shape + (-1, model.AA_COUNT))
    var_model_encoding = var_model_encoding.squeeze(0)
    log_prob = mt.seq_log_probability(var_one_hot, var_model_encoding)
    
    # make variant fitness relative to the wild type
    model_scores.append(log_prob - wt_prob)
    
model_scores = pd.Series(model_scores)
model_scores, subset['DMS_score']



In [ ]:
a = torch.Tensor([0,0,0])
b = torch.Tensor([1,2,3])
a - b

In [ ]:
spear_rho, k_recall, ndcg, roc_auc = mt.summary_stats(predictions=model_scores, actual=subset['DMS_score'], actual_binned=subset['DMS_score_bin'])
spear_rho, k_recall, ndcg, roc_auc

In [ ]:
import evoVAE.utils.seq_tools as st
from evoVAE.utils.seq_tools import GAPPY_PROTEIN_ALPHABET, calc_position_prob_matrix, calc_mean_seq_embeddings, calc_position_prob_matrix, create_euclidean_dist_matrix, plot_residue_distributions
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, List

seqs = st.read_aln_file("../data/test.aln")
encode = calc_mean_seq_embeddings(seqs)
ppm = calc_position_prob_matrix(seqs)


create_euclidean_dist_matrix(encode, plot=True)


plot_residue_distributions(encode)
#print(ppm)
#print()
#print(avgs)


In [ ]:
x = []
y = []
all = []

with open("sample_0.01.output", 'r') as file:
    lines = file.readlines()
    for line in lines:
        data = line.strip().split(',')
        print(data)
        all.append(data)

        x.append(float(data[0]))
        y.append(float(data[1]))

In [ ]:



print(all[-1])
error_plus = [(float(all[-1][1]) + float(all[-1][2])/np.sqrt(30000)) for x in range(len(x))]
error_neg = [(float(all[-1][1]) - float(all[-1][2])/np.sqrt(30000)) for x in range(len(x))]
print(error_plus)
plt.figure(figsize=(10,8))
plt.plot(x,y)
plt.plot(x, [float(all[-1][1]) for x in range(len(x))], label='true mean deviation')
plt.plot(x,error_plus, '--r', label="true mean + 1SD")
plt.plot(x,error_neg, '--r', label="true mean - 1SD")
plt.title("GFP ancestors")
plt.ylabel("Average Euclidean distance from population residue proportions", size=12)
plt.xlabel("Proportion of total population sampled")
plt.legend()

